# Overview of mosaic

<b>Objective</b><br>
To showcase the minimum number of steps<br>
required to do tertiary analysis of DNA + Protein<br>
and some of the different ways to look at the data<br>

<b>Major questions answered:</b>
1. Do we see DNA clones?<br>
2. Do we see protein cell types<br>
3. Is the differential expression significant?<br>
4. Do the clones correlate with the cell types?<br>

<b>Things not shown:</b>
1. All available methods eg. Filtering of nearby variants, variant annotation, plots

2. Discussing all methods and their options - Documented [here](https://missionbio.github.io/mosaic/)<br>

3. Systemic variations seen in protein data

### Setup

H5 files are a replacement of loom files. These are part of the DNA and protein pipeline output.
       
<i>Note: This example h5 file trimmed specifically for this analysis</i>

[Here](https://missionbio.github.io/mosaic/pages/functions/missionbio.mosaic.io.load.html#missionbio.mosaic.io.load) is the complete documentation on the load function

In [ ]:
import missionbio.mosaic as ms

sample = ms.load_example_dataset("3 cell mix")  # Use ms.load(path_to_h5) for custom h5 files

### Data Structure

[Dna](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.dna.Dna.html), [Cnv](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.cnv.Cnv.html), and [Protein](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.protein.Protein.html) are sub classes of the [_Assay](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.assay._Assay.html) class<br>
The information is stored in four ways, and the user<bR>
can change each of those<br>

    1. metadata (add_metadata / del_metadata):
        dictionary containing metrics of the assay

    2. row_attrs (add_row_attr / del_row_attr):
        dictionary which contains 'barcode' as one of
        the keys. All the values must be of the same
        length i.e. match the number of barcodes
        This is the attribute where 'label', 'pca',
        and 'umap' values are added

    3. col_attrs (add_col_attr / del_col_attr):
        dictionary which contains 'ids' as one of
        the keys. All the values must be of the same
        length i.e. match the number ids
        'ids' contains variants for DNA assays
        and anitobides for Protein assays

    4. layers (add_layer / del_layer):
        dictionary containing 'read_counts' as one of 
        the metrics. All the values have the shape
        (num barcodes) x (num ids). This is the attribute
        where 'normalized_counts' will be added

[Sample](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.sample.Sample.html) holds the [Dna](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.dna.Dna.html) and [Protein](https://missionbio.github.io/mosaic/pages/missionbio.mosaic.protein.Protein.html) information
    



In [ ]:
sample.protein

In [ ]:
sample.protein.metadata

In [ ]:
sample.protein.row_attrs

In [ ]:
sample.protein.ids()

In [ ]:
sample.dna.layers

### DNA Analysis

<b>Topcis covered</b><br>
1. Whitelist of variants
2. Manually selecting variants

#### Basic filtering

    Many filtering options are available
    use the documentation shared earlier,
    or the help() function to get the same
    information here

In [ ]:
help(sample.dna.filter_variants)

In [ ]:
# Filter variants
# This is the default insights filtering method

dna_vars = sample.dna.filter_variants()
dna_vars

In [ ]:
# Check the number of filtered variants

len(dna_vars)

#### Whitelist

    Simply appnding the whitelist to the list of filtered
    variants is sufficient to then select the variants
    using the slice notation
    
    i.e. sample.dna[{list of barcodes}, {list of ids}]

In [ ]:
whitelist = ['chr1:115256513:G/A', 'chr21:44514718:C/T']

In [ ]:
final_vars = whitelist + list(dna_vars)

In [ ]:
len(final_vars)

In [ ]:
# Selecting all cells and final variants

sample.dna = sample.dna[sample.dna.barcodes(), final_vars]

In [ ]:
# Check the shape i.e. (Number of barcodes, number of ids)
# of the final filtered dna object

sample.dna.shape

#### Manual variant selection

    Heatmaps are interactive. Clicking on it selects
    the corresponding id whose value is stored in the
    `selected_ids` attribute of the object
    
    eg. sample.dna.selected_ids

In [ ]:
# Remove `.show("jpg")` to make the plot interactive
sample.dna.stripplot(attribute='AF', colorby='GQ').show("jpg")

In [ ]:
sample.dna.heatmap(attribute='AF')

In [ ]:
if len(sample.dna.selected_ids) > 0:
    sample.dna = sample.dna.drop(sample.dna.selected_ids)

#### Clustering

    DNA has a custom clustering method called `find_clones`
    
    It projects the data on a UMAP and then performs
    dbscan to identify unique clusters, which are then
    merged in case they were formed due to missing
    information

In [ ]:
sample.dna.find_clones()

In [ ]:
sample.dna.row_attrs

In [ ]:
sample.dna.scatterplot(attribute='umap', colorby='label').show("jpg")

In [ ]:
# AF_MISSING is the same as the AF layer except that it stores the missing values as -50 instead of 0
sample.dna.heatmap('AF_MISSING').show("jpg")

#### Conclusion

    1. Basic filtering of barcodes ids demonstrated
    2. Basic DNA filtering functionality showcased

### CNV Analysis

Preliminary heatmap of CNV shows that there could be two clusters

<b>Topics covered</b>
1. Dimension reduction options and their effects

#### Observation

In [ ]:
sample.cnv.normalize_reads()
sample.cnv.heatmap(attribute='normalized_counts').show("jpg")

#### PCA options

    Here the UMAP options are kept constant
    The only parameter in PCA is the number of components

    Here we see how to determine this value, and the effect
    when we deviate from this value

In [ ]:
sample.cnv.run_pca(attribute='normalized_counts', components=6, show_plot=True)
sample.cnv.run_umap(attribute='pca', min_dist=0, n_neighbors=100, random_state=42)

#### Visualization

    The result of the dimension reduction analysis is
    visualized using a scatterplot of the umap

In [ ]:
sample.cnv.cluster(attribute='umap', method='dbscan', eps=0.55)

In [ ]:
sample.cnv.scatterplot(attribute='umap', colorby='label').show("jpg")

#### CNV Conclusion

    Given all other variables are kept constant

    1. Too many PCA components may result in merging of clusters
    2. Too few PCA component may result in splitting of clusters
    3. The appropriate number of components can be determined using the elbow plot

### Protein Analysis

<b>Topics covered</b>
1. Basic workflow
2. Custom clustering eg. selection on biaxial plot
3. Custom methods by adding layers

#### Basic workflow

In [ ]:
# Downsampling and clustering similar to CNV

sample.protein.normalize_reads('CLR')
sample.protein.run_pca(attribute='normalized_counts', components=5)
sample.protein.run_umap(attribute='pca')

sample.protein.cluster(attribute='pca', method='graph-community', k=100, random_state=42)

In [ ]:
sample.protein.heatmap(attribute='normalized_counts').show("jpg")

In [ ]:
sample.protein.scatterplot(attribute='umap', colorby='label').show("jpg")

In [ ]:
# Re cluster based on the observations from the UMAP

sample.protein.cluster(attribute='umap', method='dbscan')

In [ ]:
sample.protein.ids()[:1]

In [ ]:
# Prefered way to look at protein expression profiles

features = ["CD110"]

sample.protein.ridgeplot(
   attribute='normalized_counts',
   splitby='label',
   features=features,
).show("jpg")

In [ ]:
# UMAP with the expression for each of the selected protein overlayed
# In case of error, make sure that ids have been selected on the heatmap and shown in sample.protein.selected_ids

sample.protein.scatterplot(
   attribute='umap',
   colorby='normalized_counts',
   features=['CD34', 'CD44', 'HLA-DR'],
).show("jpg")

#### Custom clustering

    When `colorby` is not provided for any scatterplot
    the lasso tool can be used to cluster the cells
    based on the selection made

In [ ]:
# Selction on biaxial scatterplot
# The same can be done for the UMAP when labels=False is passed

sample.protein.feature_scatter(
    layer='normalized_counts',
    ids=['CD90', 'CD3']
)

#### Custom methods by adding layers

    If someone is interested in trying their methods,
    they can simply modify the appropriate layers, attributes
    and metadata to plugin their step in this workflow

In [ ]:
# Custom normalization by changing the `normalized_counts` layer

import numpy as np

log_reads = np.log10(10 + sample.protein.layers['read_counts'])
norm = np.divide(log_reads, log_reads.mean(axis=1).reshape(-1, 1))


sample.protein.add_layer('normalized_counts', norm)

    Other examples include:
    
    custom labels -> 'label' row_attr
    custom palette -> 'palette' metadata   

#### Protein Conclusion

    1. Protein analysis workflow similar to CNV
    2. Different clustering methods can result in
       different types of clusters being identified
    3. It is possible to have custom clustering for
       any scatterplot by using the lasso tool
    4. Custom analysis is possible by modifying appropriate
       layers, attributes and metadata

### Statistical Significance

    The significane of differential expression
    based on a t-test can be looked at using
    the `feature_signature` method

In [ ]:
pval, tstat = sample.protein.test_signature(attribute='normalized_counts')

In [ ]:
pval

In [ ]:
pval = pval + 10 ** -50 + pval
pvals = -np.log10(pval) * (tstat > 0)

In [ ]:
from missionbio.mosaic.plots.heatmap import Heatmap

fig = Heatmap(pvals, y_groups=pvals.index.values).draw()
fig.show("jpg")

Conclusion

    Statistical significance of the differential expression
    can be ascertained. Median values can be explored for DNA
    to determine the difference between clusters.

### Combined Visualizations

    Visualization for multiple assays at once

#### Clone vs Analyte

##### CNV

In [ ]:
sample.clone_vs_analyte('cnv')

##### Protein

In [ ]:
sample.clone_vs_analyte('protein')

In [ ]:
# Filtering protein and cnv to improve the visualization

sample.protein = sample.protein[:, ['CD3', 'CD90']]
sample.cnv = sample.cnv[:, 58:85]
sample.clone_vs_analyte('protein')

In [ ]:
# Certain clones can also be dropped, but they must be dropped from all assays
# Hence the sample object is sliced in this case
# In this case it is better to store the new sample in a separate variable

# This returns the dna barcodes with the given labels
select_bars = sample.dna.barcodes(['2', '3', '4'])

sample_subset = sample[select_bars]
sample_subset.clone_vs_analyte('protein')

In [ ]:
# The ids can also be reset to the entire set

sample.reset('cnv')
sample.reset('protein')
sample.clone_vs_analyte('protein')

#### Multi assay heatmap

In [ ]:
sample.heatmap(("dna", "protein", "cnv")).show("jpg")

# Try the following
# sample.signaturemap(("dna", "protein", "cnv"))
# sample.signaturemap(("protein", "dna", "cnv"))
# sample.signaturemap(("dna", "protein"))

### Saving

    The analysis can be saved to an h5 file.
    This final trimmed file will be much smaller than the original h5 file.
    It can be opened in Insights, or back again in Mosaic

In [ ]:
ms.save(sample, './basics.analyzed.h5', mode="w")

In [ ]:
ms.to_zip(sample, "./basics.analyzed.zip")

    Data from h5 files can be efficiently manipulated,
    visualized, and inferred using Mosaic.